# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [2]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
display(data.head(10))
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Выведенная таблица состоит из 21525 строк и 12 стобцов:

+ children — количество детей в семье - (int64)
+ days_employed — общий трудовой стаж в днях - (float64)
+ dob_years — возраст клиента в годах - (int64)
+ education — уровень образования клиента - (object)
+ education_id — идентификатор уровня образования - (int64)
+ family_status — семейное положение - (object)
+ family_status_id — идентификатор семейного положения - (int64)
+ gender — пол клиента - (object)
+ income_type — тип занятости - (object)
+ debt — имел ли задолженность по возврату кредитов - (int64)
+ total_income — ежемесячный доход - (float64)
+ purpose — цель получения кредита - (object)

В столбцах days_employed и total_income имеются пропуски.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
data[data['total_income'].isnull()] #определяем пропуски в столбце 'total_income' - ежемесячный доход

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [4]:
len(data[data['total_income'].isna()]) #считаем общее количество пропусков NaN в стобце 'total_income'

2174

In [5]:
data[data['total_income'].isna()].count() #количество пропущенных значений NaN, в целом по таблице

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

Из таблицы видим, что количество пропусков в столбцах days_employed и total_income идентично и они находятся на одних и тех же строчках. Для дальнейших операций возьмем столбец ежемесячного дохода, поскольку это более весомый аргумент в получении кредита нежели трудовой стаж, тем более, что цифры в total_income хотя бы одного порядка и одного знака.

In [6]:
median_income_type = data.groupby('income_type')['total_income'].transform('mean')
#посчитаем средние значения ежемесячного дохода в столбце total_income, согласно типу занятости income_type 

In [7]:
data['total_income'].fillna(median_income_type, inplace=True) 
#Заменим пропуски в столбце total_income средними значениями ежемесячного дохода

In [8]:
len(data[data['total_income'].isna()]) #проверим общее количество пропусков NaN в стобце 'total_income'

0

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод** 

В результате получили таблицу из 21524 строк и 11 столбцов, в процессе обработки был отброшен столбец по стажу работы, за ненадобностью. Пропуски были заменены на средние значения ежемесячного дохода типу занятости, вероятней всего пропуски были допущены при заполнении заявления заемщиком с целью сокрытия данных или же на момент подачи заявки клиент не работал.

### Замена типа данных

In [10]:
data['total_income'] = data['total_income'].astype('int64')
#округлим значения столбца ежемесячного дохода до целых чисел, с помощью метода astype()
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Табличные значения легче воспринимать когда это целые числа, а не дробные.

### Обработка дубликатов

Значения столбца education вызывают вопросы

In [11]:
data['education'].value_counts() #обнаружены неявные дубликаты, с помощью метода value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [12]:
data['education'] = data['education'].str.lower() #приведем все значения к нижнему регистру, с помощью метода string.lower()

In [13]:
data['education'].unique() #проверка

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [14]:
data.duplicated().sum() #в датафрейме обнаружено 71 явный дубликат, с помощью метода sum()

71

In [15]:
data = data.drop_duplicates().reset_index(drop = True)
#сохраним таблицу, очищенную от дубликатов, с новой индексацией
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21450,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21451,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21452,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


In [16]:
data.duplicated().sum() #проверим

0

**Вывод**

Значительное количество неявных дубликатов было представлено в столбце, уровень образования клиента, также было выявлено 71 явный дубликат. После зачистки дубликатов не обнаружено, а количество строк в таблице уменьшилось до 21454.

### Лемматизация

In [17]:
from pymystem3 import Mystem 
m = Mystem() 

In [18]:
from collections import Counter
print(Counter(data['purpose'])) 

Counter({'свадьба': 791, 'на проведение свадьбы': 768, 'сыграть свадьбу': 765, 'операции с недвижимостью': 675, 'покупка коммерческой недвижимости': 661, 'операции с жильем': 652, 'покупка жилья для сдачи': 651, 'операции с коммерческой недвижимостью': 650, 'покупка жилья': 646, 'жилье': 646, 'покупка жилья для семьи': 638, 'строительство собственной недвижимости': 635, 'недвижимость': 633, 'операции со своей недвижимостью': 627, 'строительство жилой недвижимости': 624, 'покупка недвижимости': 621, 'покупка своего жилья': 620, 'строительство недвижимости': 619, 'ремонт жилью': 607, 'покупка жилой недвижимости': 606, 'на покупку своего автомобиля': 505, 'заняться высшим образованием': 496, 'автомобиль': 494, 'сделка с подержанным автомобилем': 486, 'на покупку подержанного автомобиля': 478, 'автомобили': 478, 'свой автомобиль': 478, 'на покупку автомобиля': 471, 'приобретение автомобиля': 461, 'дополнительное образование': 460, 'сделка с автомобилем': 455, 'высшее образование': 452, 'об

In [19]:
data['purpose'].value_counts() #посмотрим на цели кредита

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Среди целей кредита выделяются четыре направления:  
+ свадьба
+ недвижимость, жилье
+ автомобиль
+ образование

In [20]:
def lemmatization (purpose): #зададим группы целей для получения кредита
              
    lemma = m.lemmatize(purpose) 

    if ('недвижимость' in lemma) or ('жилье' in lemma):
        lemma_name = 'недвижимость'
        return lemma_name
    
    elif 'свадьба' in lemma:
        lemma_name = 'свадьба'
        return lemma_name
     
    elif 'автомобиль' in lemma:
        lemma_name = 'автомобиль'
        return lemma_name
   
    else:
        lemma_name = 'образование'
        return lemma_name

In [21]:
lemma_purpose = data['purpose'].apply(lemmatization)
#берём значения столбца датафрейма и применяем к ним функцию со своим аргументом

In [22]:
data['purpose'] = lemma_purpose #возвращаем в датафрейм значения целей кредита

In [23]:
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость


In [24]:
display(data['purpose'].value_counts())

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose, dtype: int64

In [25]:
Counter(data['purpose'].apply(m.lemmatize).sum()) 

Counter({'недвижимость': 10811,
         '\n': 21454,
         'автомобиль': 4306,
         'образование': 4013,
         'свадьба': 2324})

Попробуем найти **артефакты** в других столбцах.

In [26]:
data['children'].value_counts() #посмотрим количество детей.
#из .value_counts() видно что у 47 человек количество детей ошибочно записано с минусом

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [27]:
data['children'] = data['children'].abs() #возьмем число детей по модулю чтобы исключить отрицательные значения
#следующей опечаткой является число 20, врядли у 76 человек так много детей, скорей всего количество детей двое.
data['children'] = data['children'].replace(20,2)

data['children'].value_counts() #проверим

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [28]:
data['dob_years'].unique() #посмотрим значения возраста.

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [29]:
data=data.drop(data[data['dob_years']==0].index) #удаляем значение 0
data['dob_years'].unique() #проверим

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

In [30]:
data['education_id'].unique() #проверим идентификатор уровня образования

array([0, 1, 2, 3, 4])

Ничего особенного в этом столбце нет
тоже самое со всеми оставшимися столбцами, кроме пола клиента

In [31]:
data['gender'].value_counts() #проверим пол клиента

F      14102
M       7250
XNA        1
Name: gender, dtype: int64

In [32]:
data=data.drop(data[data['gender']=='XNA'].index) #удалим "средний род" и перепроверим
data['gender'].unique()

array(['F', 'M'], dtype=object)

**Вывод**

Провели лемматизацию целей кредита, то есть выделили ключевые слова относящиееся к определенной категории кредита, а затем удалили оставшиеся огрехи таблицы, которые могли возникнуть при ошибочном введении данных.

### Категоризация данных

In [33]:
data.sort_values(by = 'dob_years') #отсортируем данные по возрасту заемщиков
#минимальный возраст 19 лет, а максимальный 75

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
8307,0,-556.088508,19,среднее,1,Не женат / не замужем,4,F,сотрудник,0,68524,образование
4050,0,NaN,19,среднее,1,гражданский брак,1,M,сотрудник,0,161380,свадьба
15624,0,-948.225743,19,неоконченное высшее,2,гражданский брак,1,F,компаньон,0,167211,автомобиль
2725,0,-115.876036,19,среднее,1,Не женат / не замужем,4,F,компаньон,0,131967,недвижимость
12027,0,-1020.183130,19,среднее,1,Не женат / не замужем,4,M,сотрудник,1,93343,недвижимость
...,...,...,...,...,...,...,...,...,...,...,...,...
3458,0,344623.836105,74,среднее,1,женат / замужем,0,M,пенсионер,0,54754,недвижимость
12297,0,-1729.632531,74,среднее,1,женат / замужем,0,M,компаньон,0,124830,недвижимость
4891,0,341528.126150,74,высшее,0,женат / замужем,0,F,пенсионер,0,134935,недвижимость
2557,0,372861.103965,74,среднее,1,женат / замужем,0,F,пенсионер,0,42927,автомобиль


In [34]:
def age_group(age): #разобьем заемщиков по возрастам
        
    if 19 <= age <= 30:
        return 'молодые'

    if 31 <= age <=59:
        return 'взрослые'

    return 'пенсионеры' 


In [35]:
age_group(38) #проверим работоспособность функции


'взрослые'

In [36]:
data['dob_years'] = data['dob_years'].apply(age_group) #применим функцию к столбцу с возрастом с помощью метода apply()

In [37]:
data['dob_years'].value_counts() #количество людей в возрастных группах

взрослые      15136
молодые        3716
пенсионеры     2500
Name: dob_years, dtype: int64

In [38]:
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,взрослые,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,-4024.803754,взрослые,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,-5623.422610,взрослые,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,-4124.747207,взрослые,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,340266.072047,взрослые,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529.316663,взрослые,среднее,1,гражданский брак,1,F,компаньон,0,224791,недвижимость
21450,0,343937.404131,пенсионеры,среднее,1,женат / замужем,0,F,пенсионер,0,155999,автомобиль
21451,1,-2113.346888,взрослые,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21452,3,-3112.481705,взрослые,среднее,1,женат / замужем,0,M,сотрудник,1,244093,автомобиль


In [39]:
data.sort_values(by = 'total_income') #отсортируем заемщиков по уровню дохода

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14555,0,359219.059341,взрослые,среднее,1,женат / замужем,0,F,пенсионер,1,20667,недвижимость
12983,0,369708.589113,взрослые,среднее,1,гражданский брак,1,M,пенсионер,0,21205,образование
16138,1,-3642.820023,взрослые,среднее,1,женат / замужем,0,M,сотрудник,0,21367,автомобиль
1598,0,359726.104207,пенсионеры,среднее,1,гражданский брак,1,M,пенсионер,0,21695,свадьба
14247,0,346602.453782,пенсионеры,среднее,1,женат / замужем,0,F,пенсионер,0,21895,недвижимость
...,...,...,...,...,...,...,...,...,...,...,...,...
17137,0,-5734.127087,взрослые,высшее,0,гражданский брак,1,M,компаньон,0,1711309,свадьба
20742,0,-4719.273476,пенсионеры,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1715018,недвижимость
9159,1,-5248.554336,взрослые,среднее,1,гражданский брак,1,M,сотрудник,0,1726276,образование
19548,1,-2577.664662,взрослые,высшее,0,женат / замужем,0,M,компаньон,1,2200852,недвижимость


Из таблицы видно, что доход лежит в интервале от 20 тысяч до 2,2 млн руб.

In [40]:
def total_income_group(income): #разобьем заемщиков по уровню дохода
        
    if 20000 <= income <= 50000:
        return 'низкий'

    if 50001 <= income <=120000:
        return 'средний'
   
    if 120001 <= income <=210000:
        return 'высокий'

    return 'очень высокий' 

In [41]:
total_income_group(160000) #проверим работоспособность функции

'высокий'

In [42]:
data['total_income'] = data['total_income'].apply(total_income_group) 
#применим функцию к столбцу с доходом с помощью метода apply()

In [43]:
data['total_income'].value_counts() #количество людей с разным доходом

высокий          10064
средний           6438
очень высокий     4480
низкий             370
Name: total_income, dtype: int64

In [44]:
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,взрослые,высшее,0,женат / замужем,0,F,сотрудник,0,очень высокий,недвижимость
1,1,-4024.803754,взрослые,среднее,1,женат / замужем,0,F,сотрудник,0,средний,автомобиль
2,0,-5623.422610,взрослые,среднее,1,женат / замужем,0,M,сотрудник,0,высокий,недвижимость
3,3,-4124.747207,взрослые,среднее,1,женат / замужем,0,M,сотрудник,0,очень высокий,образование
4,0,340266.072047,взрослые,среднее,1,гражданский брак,1,F,пенсионер,0,высокий,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529.316663,взрослые,среднее,1,гражданский брак,1,F,компаньон,0,очень высокий,недвижимость
21450,0,343937.404131,пенсионеры,среднее,1,женат / замужем,0,F,пенсионер,0,высокий,автомобиль
21451,1,-2113.346888,взрослые,среднее,1,гражданский брак,1,M,сотрудник,1,средний,недвижимость
21452,3,-3112.481705,взрослые,среднее,1,женат / замужем,0,M,сотрудник,1,очень высокий,автомобиль


**Вывод**

Были категоризированны два столбца данных: возраст заемщика и его доход, поскольку эти два фактора оказывают слильное влияние на одобрение кредита. 

## Шаг 3. Ответьте на вопросы

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Нас интересуют столбцы: 'children' (количество детей в семье) и 'debt' (задолжность по возврату кредита)  

In [45]:
data['debt'].value_counts() #задолжность по возрврату имеет всего два значения - либо она есть, либо ее нет

0    19619
1     1733
Name: debt, dtype: int64

In [46]:
data['children'].value_counts() #количество детей у заемщиков

0    14021
1     4839
2     2114
3      328
4       41
5        9
Name: children, dtype: int64

In [47]:
data_debt_children = data.groupby('children').agg({'debt':['count','sum']}) #агрегирующая функция
display(data_debt_children)

debt      
          count   sum
children             
0         14021  1058
1          4839   442
2          2114   202
3           328    27
4            41     4
5             9     0

In [48]:
data_debt_children_procent = data_debt_children['debt']['sum']/data_debt_children['debt']['count']*100
data_debt_children_procent.sort_values()
#найдем долю невозвращенных кредитов у людей с разным количеством детей

children
5    0.000000
0    7.545824
3    8.231707
1    9.134119
2    9.555345
4    9.756098
dtype: float64

In [49]:
data_debt_children['data_debt_children_procent'] = data_debt_children_procent = data_debt_children['debt']['sum']/data_debt_children['debt']['count']*100
display(data_debt_children)

debt       data_debt_children_procent
          count   sum                           
children                                        
0         14021  1058                   7.545824
1          4839   442                   9.134119
2          2114   202                   9.555345
3           328    27                   8.231707
4            41     4                   9.756098
5             9     0                   0.000000

**Вывод**

Зависимость присутствует, меньше всего долгов у тех людей, которые не имеют детей - 7,5%, как только в семье появляется пополнение, оно оттягивает на себя часть доходов и долги растут, но в таблице имеется 9 человек у кого в семье по 5 детей, такие семьи оказались без долгов, думаю это связано с тем что их всего 9 человек из 21 тысячи заемщиков, просто совпадение.  

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Нас интересуют столбцы: 'family_status' (семейное положение) и 'debt' (задолжность по возврату кредита)  

In [50]:
data['family_status'].value_counts() #количество людей с разным семейным положением

женат / замужем          12290
гражданский брак          4129
Не женат / не замужем     2794
в разводе                 1185
вдовец / вдова             954
Name: family_status, dtype: int64

In [51]:
data_debt_family_status = data.groupby('family_status').agg({'debt':['count','sum']}) #агрегирующая функция
display(data_debt_family_status)

debt     
                       count  sum
family_status                    
Не женат / не замужем   2794  273
в разводе               1185   85
вдовец / вдова           954   62
гражданский брак        4129  386
женат / замужем        12290  927

In [52]:
data_debt_family_status_procent = data_debt_family_status['debt']['sum']/data_debt_family_status['debt']['count']*100
data_debt_family_status_procent.sort_values()
#найдем долю невозвращенных кредитов у людей с разным семейным положением

family_status
вдовец / вдова           6.498952
в разводе                7.172996
женат / замужем          7.542718
гражданский брак         9.348511
Не женат / не замужем    9.770938
dtype: float64

In [53]:
data_debt_family_status['data_debt_family_status_procent'] = data_debt_family_status['debt']['sum']/data_debt_family_status['debt']['count']*100
display(data_debt_family_status)

debt      data_debt_family_status_procent
                       count  sum                                
family_status                                                    
Не женат / не замужем   2794  273                        9.770938
в разводе               1185   85                        7.172996
вдовец / вдова           954   62                        6.498952
гражданский брак        4129  386                        9.348511
женат / замужем        12290  927                        7.542718

**Вывод**

Наиболее рисковые для банка группы: люди, проживающие в гражданском браке и не женатые/не замужние. Люди, проживающие в гражданском браке, часто расходятся и по отдельности не могут гасить задолжности, а не женатые/не замужние могут пропасть из виду, накопленные долги нельзя передать родителям, поскольку их дети достигшие совершеннолетние считаются полностью самостоятельными. Средними показателями обладают женатые пары - их больше всего.

- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Нас интересуют столбцы: 'total_income' (ежемесячный доход) и 'debt' (задолжность по возврату кредита)  

In [54]:
data['total_income'].value_counts() #количество людей с разным доходом

высокий          10064
средний           6438
очень высокий     4480
низкий             370
Name: total_income, dtype: int64

In [55]:
data_debt_total_income = data.groupby('total_income').agg({'debt':['count','sum']}) #агрегирующая функция
display(data_debt_total_income)

debt     
               count  sum
total_income             
высокий        10064  871
низкий           370   23
очень высокий   4480  314
средний         6438  525

In [56]:
data_debt_total_income_procent = data_debt_total_income['debt']['sum']/data_debt_total_income['debt']['count']*100
data_debt_total_income_procent.sort_values()
#найдем долю невозвращенных кредитов у людей с разным доходом

total_income
низкий           6.216216
очень высокий    7.008929
средний          8.154706
высокий          8.654610
dtype: float64

In [57]:
data_debt_total_income['data_debt_total_income_procent'] = data_debt_total_income['debt']['sum']/data_debt_total_income['debt']['count']*100
display(data_debt_total_income)

debt      data_debt_total_income_procent
               count  sum                               
total_income                                            
высокий        10064  871                       8.654610
низкий           370   23                       6.216216
очень высокий   4480  314                       7.008929
средний         6438  525                       8.154706

**Вывод**

В данном случае меньше всего долгов у людей с низким уровнем дохода, это просто объяснить - они не берут кредиты вовсе.
Самое большое количество долгов накопилось у заемщиков со средним и высоким уровнем дохода, это как раз основная масса людей, которые обращаются в банки. Люди со сверхвысоким доходом реже обращаются в банки, поскольку заработка и накопленных средств им хватает.

- **Как разные цели кредита влияют на его возврат в срок?**

Нас интересуют столбцы: 'purpose' (цель получения кредита) и 'debt' (задолжность по возврату кредита)  

In [58]:
data['purpose'].value_counts() #количество людей с разными целями по кредиту

недвижимость    10763
автомобиль       4284
образование      3995
свадьба          2310
Name: purpose, dtype: int64

In [59]:
data_debt_purpose = data.groupby('purpose').agg({'debt':['count','sum']}) #агрегирующая функция
display(data_debt_purpose)

debt     
              count  sum
purpose                 
автомобиль     4284  400
недвижимость  10763  779
образование    3995  370
свадьба        2310  184

In [60]:
data_debt_purpose_procent = data_debt_purpose['debt']['sum']/data_debt_purpose['debt']['count']*100
data_debt_purpose_procent.sort_values()
#найдем долю невозвращенных кредитов у людей с разными целями по кредиту

purpose
недвижимость    7.237759
свадьба         7.965368
образование     9.261577
автомобиль      9.337068
dtype: float64

In [61]:
data_debt_purpose['data_debt_purpose_procent'] = data_debt_purpose['debt']['sum']/data_debt_purpose['debt']['count']*100
display(data_debt_purpose)

debt      data_debt_purpose_procent
              count  sum                          
purpose                                           
автомобиль     4284  400                  9.337068
недвижимость  10763  779                  7.237759
образование    3995  370                  9.261577
свадьба        2310  184                  7.965368

**Вывод**

Как видим из полученных занчений, наибольшими долгами обладают люди, покупающие в кредит автомобили, наиболее надежными заемщиками являются люди, берущие ипотеку.

## Шаг 4. Общий вывод

В выполненном проекте, были изучены и обработанны данные, полученные от заказчика. Наиболее надежным для банка клиентом будет условно, женатый человек, со средним или высоким доходом, не имеющий детей, и планирующий обзавестись новой недвижимостью. 